# Reconhecimento de caracteres alfabéticos utilizando redes neurais convolucionais

Este notebook apresenta o treinamento de uma rede neural convolucional (CNN) para reconhecimento de caracteres alfabéticos manuscritos. O dataset utilizado foi extraído do Kaggle no formato CSV e passou por pré-processamento, incluindo binarização dos pixels para facilitar o aprendizado do modelo. A base de dados foi dividida em conjuntos de treino e teste, garantindo que a validação fosse feita com dados inéditos. Além disso, foi aplicada a técnica de aumento de dados, com rotações e deslocamentos aleatórios, para tornar o modelo mais robusto.

A arquitetura da rede conta com múltiplas camadas convolucionais, regularização L2 para evitar overfitting, camadas de BatchNormalization para estabilização do aprendizado e Dropout para melhorar a generalização. O treinamento foi conduzido utilizando callbacks como EarlyStopping e ReduceLROnPlateau para otimização. Após o treinamento, o modelo alcançou uma precisão de 99,18% no conjunto de testes, demonstrando sua eficiência no reconhecimento de caracteres manuscritos.

## Importando as bibliotecas necessárias

Nesse notebook as bibliotecas utilizadas serão:

* sklearn - Para geração do dataset de treino
* Tensorflow - Para criação e manipulação das redes neurais utilizadas
* Pandas - Para lidar com os arquivos .csv que compoem o dataset
* Numpy - Para funções matemáticas - nesse caso: a random

In [ ]:
import os 
import keras
import numpy as np
import tensorflow as tf

import pandas as pd
from keras.preprocessing.image import ImageDataGenerator 
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split

import matplotlib.pyplot as plt

from tensorflow.keras.regularizers import l2

from keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Activation, BatchNormalization

from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau

## Abrindo o dataset

O dataset utilizado foi obtido no formato de .csv no [Kaggle](https://www.kaggle.com/datasets/sachinpatel21/az-handwritten-alphabets-in-csv-format).
Ele será aberto utilizando o Pandas.

In [ ]:

file_path = '/kaggle/input/az-handwritten-alphabets-in-csv-format/A_Z Handwritten Data.csv'

names = ['class']
for id in range(1,785):
    names.append(id)

df = pd.read_csv(file_path,header=None, names=names)

class_mapping = {}
alphabets = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
for i in range(len(alphabets)):
    class_mapping[i] = alphabets[i]
    
df['class'].map(class_mapping).unique()

y_full = df.pop('class')
x_full = df.to_numpy().reshape(-1,28,28, 1)

## Split do dataset

Os samples presentes no dataframe serão divididos em 2 conjuntos: o de treino e o de teste.

Os samples presentes no dataset de testes não serão apresentados durante o treinamento da rede neural para que a sua precisão possa ser validada com dados inéditos - simulando o ambiente real de produção.

In [ ]:
val_split = 0.2

splitter = StratifiedShuffleSplit(n_splits=3,test_size=val_split)

for train_ids, test_ids in splitter.split(x_full, y_full):
    X_train_full, y_train_full = x_full[train_ids], y_full[train_ids].to_numpy()
    X_test, y_test = x_full[test_ids], y_full[test_ids].to_numpy()

X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=val_split)

## Pre-processando as imagens

Para facilitar o aprendizado do modelo, as imagens serão pre-processadas.
O pré-processamento consiste na aplicação do filtro de threshold, binarizando os valores, em outras palavras:

Se o valor do pixel > 125, pixel = 1
Se o valor do pixel < 125, pixel = 0

Isso acontece pois os valores dos pixels em escala de cinza (entre 0 e 255) podem confudir a rede durante seu aprendizado, portanto as imagens são "simplificadas".

### Imagens antes do pre-processamento

In [ ]:
plt.figure(figsize=(15,8))
for i in range(1, 11):
    
    id = np.random.randint(len(X_train))
    image, label = tf.squeeze(X_train[id]), class_mapping[int(y_train[id])]
    
    plt.subplot(2,5,i)
    plt.imshow(image, cmap='binary')
    plt.title(label)
    plt.axis('off')
    
plt.tight_layout()
plt.show()

In [ ]:
X_train = (X_train > 125)
X_test = (X_test > 125) 
X_valid = (X_valid > 125) 

### Imagens após o pré-processamento

In [ ]:
plt.figure(figsize=(15,8))
for i in range(1, 11):
    
    id = np.random.randint(len(X_train))
    image, label = tf.squeeze(X_train[id]), class_mapping[int(y_train[id])]
    
    plt.subplot(2,5,i)
    plt.imshow(image, cmap='binary')
    plt.title(label)
    plt.axis('off')
    
plt.tight_layout()
plt.show()

# Aumento de dados

Para o treino, também será usado uma técnica de aumento de dados, onde as imagens serão aleatoriamente rotacionadas em 10 graus para ambas as direções, além de ampliadas, movidas e diminuidas em 10%.

Ao proporcionar mais dados para treino, essa técnica permite com que o modelo aprenda em cima de dados irregulares e diferentes dos previamente vistos, assim aumentando seu reconhecimento dos padrões dos caracteres. 

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center = False,
        samplewise_center = False,
        featurewise_std_normalization = False,
        samplewise_std_normalization = False,
        zca_whitening = False,
        rotation_range = 10,
        zoom_range = 0.1, 
        width_shift_range = 0.1,  
        height_shift_range = 0.1, 
        horizontal_flip = False,  
        vertical_flip = False,
        validation_split=val_split
) 

datagen.fit(X_train)

# Arquitetura do modelo

O modelo de redes neurais é composto por camadas convolucionais. 
Em algumas camadas foi utilizado a Regularização L2 para evitar o overfit.
Outras tecnicas para evitar o overfit também foram utilizadas, como camadas Dropout e o callback de ReduceLROnPlateau, que reduz a taxa de aprendizado da rede caso esta fique estagnada.

Também existem camadas de BatchNormalization, responsáveis por normalizar as ativações das camadas anteriores, permitindo com que as camadas aprendam de forma mais indenpendente e não sejam tão influenciadas pelas ativações das camadas anteriores. 


In [ ]:
model = Sequential([
    Conv2D(filters = 32, kernel_size = 5, strides = 1, activation = 'relu', input_shape = (28,28,1), kernel_regularizer=l2(0.0005), name = 'convolution_1'),
    Conv2D(filters = 32, kernel_size = 5, strides = 1, name = 'convolution_2', use_bias=False),
    
    BatchNormalization(name = 'batchnorm_1'),
        
    Activation("relu"),
    MaxPooling2D(pool_size = 2, strides = 2, name = 'max_pool_1'),
    Dropout(0.25, name = 'dropout_1'),
        
    Conv2D(filters = 64, kernel_size = 3, strides = 1, activation = 'relu', kernel_regularizer=l2(0.0005), name = 'convolution_3'),
        
    Conv2D(filters = 64, kernel_size = 3, strides = 1, name = 'convolution_4', use_bias=False),
        
    BatchNormalization(name = 'batchnorm_2'),
        
    Activation("relu"),
    MaxPooling2D(pool_size = 2, strides = 2, name = 'max_pool_2'),
    
    Dropout(0.25, name = 'dropout_2'),
    Flatten(name = 'flatten'),
        
    Dense(units = 256, name = 'fully_connected_1', use_bias=False),
        
    BatchNormalization(name = 'batchnorm_3'),
    
    Activation("relu"),
        
    Dense(units = 128, name = 'fully_connected_2', use_bias=False),
        
    BatchNormalization(name = 'batchnorm_4'),
        
    Activation("relu"),
    
    Dense(units = 84, name = 'fully_connected_3', use_bias=False),
        
    BatchNormalization(name = 'batchnorm_5'),
        
    Activation("relu"),
    
    Dropout(0.25, name = 'dropout_3'),
    
    Dense(26, activation='softmax')
])

In [ ]:
model.compile(
     loss='sparse_categorical_crossentropy',
     optimizer='adam',
     metrics=['accuracy']
)

In [ ]:
cbs = [
    EarlyStopping(patience=3, restore_best_weights=True), 
    ModelCheckpoint("Model-v1.h5", save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor = 0.2, patience = 2)
]

In [ ]:
model.fit(
     datagen.flow(X_train, y_train, batch_size=64,subset='training'),
     validation_data=(X_valid, y_valid),
     epochs=50,
     callbacks=cbs
)

# Testes

O conjunto de testes foi composto por 2328 samples

In [ ]:
model.evaluate(X_test,y_test) #[0.035956788808107376, 0.9918245077133179]

Após o treinamento, a rede teve um desempenho de 99.18% de precisão no conjunto de testes 